<a href="https://colab.research.google.com/github/choarauc/form_ch/blob/main/Assurance_Prediction_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<hr style="border-width:2px;border-color:#75DFC1">
<center><h1>Deep-Learning - Modules complémentaires</h1></center>
<center><h2>Régression sur des données structurées</h2></center>
<hr style="border-width:2px;border-color:#75DFC1">


Le but de cet exercice est de s'entraîner à faire une régression sur des données structurées à l'aide d'un réseau de neurones dense.

 Nous allons travailler avec la base de données **```insurance.csv```**, qui contient les informations personnelles de clients d'une assurance, ainsi que les charges qu'ils ont à payer.

> La structure de l'exercice est la suivante :
>> I - [Préparation du dataset](#preparation)
>>
>>
>> II - [Régression linéaire multiple classique](#linear)
>>
>>
>> III - [Régression à l'aide d'un réseau de neurones dense](#dnn)
>>
>>
>> IV - [Comparaison des résultats obtenus avec les deux techniques](#comparaison)

- Exécuter les cellules ci-dessous pour importer le dataset et les modules nécessaires à l'exercice.

In [1]:
link = 'https://raw.githubusercontent.com/choarauc/dataset/main/insurance.csv'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LinearRegression

from tensorflow.keras import callbacks
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

from sklearn.metrics import r2_score

# <a name="preparation"></a> I - Préparation du dataset

- (a) Charger le jeu de données **```insurance.csv```** dans un DataFrame ```df```.

- (b) Afficher la shape de ```df``` ainsi que ses 5 premières lignes.

In [3]:
#@title
df = pd.read_csv(link)

In [ ]:
#@title
print("Nombre de lignes dans le dataset : {}".format(df.shape[0]))
print("Nombre de colonnes dans le dataset : {}".format(df.shape[1]))

df.head()

- (c) Vérifier que les colonnes de ```df``` sont du bon type, et traiter les valeurs manquantes s'il y en a.

In [ ]:
#@title
df.info()

In [ ]:
#@title
df['children'] = df['children'].astype(str)

In [ ]:
#@title
df.isna().sum()

- (d) Créer une visualisation pertinente pour chaque colonne de ```df```.

- (e) Supprimer les colonnes qui ne vous semblent pas impacter les charges à payer.

In [ ]:
#@title
charges_sex = df.groupby('sex').mean()['charges']
charges_children = df.groupby('children').mean()['charges']
charges_smoke = df.groupby('smoker').mean()['charges']
charges_region = df.groupby('region').mean()['charges']

plt.figure(figsize=(16, 10))

plt.subplot(221)
plt.bar(charges_sex.index, charges_sex.values, color='blueviolet')
plt.title("Prix moyen de l'assurance en fonction du sex")
plt.xlabel("Sex")
plt.ylabel("Prix moyen de l'assurance")

plt.subplot(222)
plt.bar(charges_children.index, charges_children.values, color='blueviolet')
plt.title("Prix moyen de l'assurance en fonction du nombre d'enfants")
plt.xlabel("Nombre d'enfants")
plt.ylabel("Prix moyen de l'assurance")

plt.subplot(223)
plt.bar(charges_smoke.index, charges_smoke.values, color='blueviolet')
plt.title("Prix moyen de l'assurance pour les fumeurs et non fumeurs")
plt.xlabel("Fumeur ?")
plt.ylabel("Prix moyen de l'assurance")

plt.subplot(224)
plt.bar(charges_region.index, charges_region.values, color='blueviolet')
plt.title("Prix moyen de l'assurance en fonction de la région")
plt.xlabel("Région")
plt.ylabel("Prix moyen de l'assurance");

In [ ]:
#@title
# Variation très faible pour la région, on peut supprimer cette colonne
df = df.drop(labels='region', axis=1)

In [ ]:
#@title
def lissage(x,y,p):
    x_lisse=[]
    y_lisse=[]
    for i in range(p,len(x)-p):
        x_lisse.append(x[i])
    for i in range(p,len(y)-p):
        val=0
        for k in range(2*p):
            val+=y[i-p+k]
        y_lisse.append(val/2/p)
    return x_lisse, y_lisse

In [ ]:
#@title
charges_age = df.groupby('age').mean()['charges']
charges_bmi = df.groupby('bmi').mean()['charges']

plt.figure(figsize=(16, 5))

plt.subplot(121)
x = charges_age.index
y = charges_age.values
x_lisse, y_lisse = lissage(x, y, 3)
plt.plot(x, y, color='turquoise', linewidth=0.7, linestyle='dashed', label='actual')
plt.plot(x_lisse, y_lisse, color='blueviolet', linewidth=2, label='tendance')
plt.xlim(20, 60)
plt.xlabel('Age')
plt.ylabel("Prix moyen de l'assurance")
plt.title("Prix moyen de l'assurance en fonction de l'age")
plt.legend()

plt.subplot(122)
x = charges_bmi.index
y = charges_bmi.values
x_lisse, y_lisse = lissage(x, y, 8)
plt.plot(x, y, color='turquoise', linewidth=0.7, linestyle='dashed', label='actual')
plt.plot(x_lisse, y_lisse, color='blueviolet', linewidth=2, label='tendance')
plt.xlim(17, 48)
plt.xlabel('BMI')
plt.ylabel("Prix moyen de l'assurance")
plt.title("Prix moyen de l'assurance en fonction du BMI")
plt.legend();

- (f) Finir de préparer les données.
> Cette partie regroupe :
>> - L'encodage des variables
>>
>>
>> - la séparation des variables explicatives de la variable cible
>>
>>
>> - La création d'un jeu de test et d'entraînement
>>
>>
>> - Le scaling de ```X_train```, ```X_test``` **et de ```y_train```, ```y_test```**


In [ ]:
#@title
df['sex'] = df['sex'].replace({'female' : 0, 'male' : 1})
df['smoker'] = df['smoker'].replace({'no' : 0, 'yes' : 1})
df = pd.get_dummies(df)

In [ ]:
#@title
df.head()

In [ ]:
#@title
X = df.drop(labels='charges', axis=1)
y = df['charges'].values

In [ ]:
#@title
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
#@title
scaler_X = MinMaxScaler()

scaler_X.fit(X_train)
X_train_scaled = pd.DataFrame(scaler_X.transform(X_train), index=X_train.index, columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler_X.transform(X_test), index=X_test.index, columns=X_test.columns)

In [ ]:
#@title
scaler_y = MinMaxScaler()

y_train=np.reshape(y_train, (-1,1))
y_test=np.reshape(y_test, (-1,1))

scaler_y.fit(y_train)
y_train_scaled = scaler_y.transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

# II - <a name="linear"></a> Régression linéaire multiple classique

- (a) Instancier un modèle de régression linéaire.

- (b) Fit le modèle aux données d'entraînement.

- (c) Effectuer une prédiction à partir des données de test (attention au scaling).

In [ ]:
#@title
lr = LinearRegression()
lr.fit(X_train_scaled, y_train_scaled)

In [ ]:
#@title
y_pred_lr = scaler_y.inverse_transform(lr.predict(X_test_scaled))

# III - <a name="dnn"></a> Régression à l'aide d'un réseau de neurones dense

- (a) Instancier un réseau de neurones dense de l'architecture de votre choix (attention à la couche de sortie, il s'agit d'un problème de régression).

- (b) Compiler le modèle avec une métrique appropriée.

- (c) Entraîner le modèle avec les paramètres de votre choix. On pourra également définir des callbacks.

- (d) Effectuer une prédiction à partir des données de test (attention au scaling).

In [ ]:
#@title
input_shape = (X_train_scaled.shape[1],)

In [ ]:
#@title
model = Sequential()
model.add(Dense(units=10, input_shape=input_shape, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=2048, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='linear'))

In [ ]:
#@title
early_stopping = callbacks.EarlyStopping(monitor = 'val_loss',
                        patience = 20,
                        mode = 'min',
                        restore_best_weights = True)

lr_plateau = callbacks.ReduceLROnPlateau(monitor = 'val_loss',
                            patience=5,
                            factor=0.8,
                            verbose=2,
                            mode='min')

In [ ]:
#@title
model.compile(loss='mae', optimizer='adam', metrics=['mse','mae'])

In [ ]:
#@title
history = model.fit(X_train_scaled, y_train_scaled, epochs = 200, batch_size = 30, validation_split = 0.2, callbacks = [early_stopping,lr_plateau])

In [ ]:
#@title
plt.figure(figsize=(12,4))
plt.subplot(121)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Perte du modèle par epoch')
plt.ylabel('perte')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='right')

plt.subplot(122)
plt.plot(history.history['mse'])
plt.plot(history.history['val_mse'])
plt.title('Erreur du modèle par epoch')
plt.ylabel('Erreur quadratique moyenne')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='right');

In [ ]:
#@title
y_pred_dnn = scaler_y.inverse_transform(model.predict(X_test_scaled))

# IV - <a name="comparaison"></a> Comparaison des résultats obtenus avec les deux techniques

- (a) Comparer les résultats obtenus avec les deux modèles (utiliser le ```r2_score``` par exemple).

In [ ]:
#@title
result = pd.DataFrame()
result['y_test'] = [y_test[i][0] for i in range(len(y_test))]
result['y_pred_lr'] = [y_pred_lr[i][0] for i in range(len(y_pred_lr))]
result['y_pred_dnn'] = [y_pred_dnn[i][0] for i in range(len(y_pred_dnn))]
result.head(10)

In [ ]:
#@title
print("r2_score du modèle lr : {}".format(r2_score(y_test, y_pred_lr)))
print("r2_score du modèle dnn : {}".format(r2_score(y_test, y_pred_dnn)))